# Procesamiento de TMY – versión notebook


In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os

def smooth_data(data, window_size=5):
    """
    Aplica un filtro de media móvil a los datos.
    
    Args:
        data (array): Datos a suavizar
        window_size (int): Tamaño de la ventana para el suavizado
    
    Returns:
        array: Datos suavizados
    """
    return np.convolve(data, np.ones(window_size)/window_size, mode='same')

def process_tmy_file(input_file, output_file, target_year=2014):
    """
    Procesa un archivo TMY y genera un reporte de calidad.
    
    Args:
        input_file (str): Ruta del archivo de entrada
        output_file (str): Ruta del archivo de salida
        target_year (int): Año al que se ajustarán todas las fechas
    """
    print(f"\nProcesando archivo: {input_file}")
    
    # Leer los datos principales
    df = pd.read_csv(input_file)
    
    # Reporte de calidad inicial
    calidad_inicial = []
    calidad_inicial.append(f"Archivo procesado: {input_file}\n")
    calidad_inicial.append(f"Total de registros: {len(df)}\n")
    calidad_inicial.append("Valores nulos por columna (antes de limpieza):\n")
    calidad_inicial.append(str(df.isnull().sum()))
    calidad_inicial.append("\nEstadísticas descriptivas (antes de limpieza):\n")
    calidad_inicial.append(str(df.describe()))
    
    # Guardar copia para comparar después
    df_limpio = df.copy()
    
    # Cambiar el año a 2014
    df_limpio['Year'] = target_year
    
    # Verificar y corregir datos
    df_limpio['GHI'] = pd.to_numeric(df_limpio['GHI'], errors='coerce').clip(lower=0, upper=1200)
    df_limpio['DNI'] = pd.to_numeric(df_limpio['DNI'], errors='coerce').clip(lower=0, upper=1200)
    df_limpio['DHI'] = pd.to_numeric(df_limpio['DHI'], errors='coerce').clip(lower=0, upper=400)
    df_limpio['Tdry'] = pd.to_numeric(df_limpio['Tdry'], errors='coerce').clip(lower=-20, upper=50)
    df_limpio['Tdew'] = pd.to_numeric(df_limpio['Tdew'], errors='coerce').clip(lower=-25, upper=30)
    df_limpio['RH'] = pd.to_numeric(df_limpio['RH'], errors='coerce').clip(lower=0, upper=100)
    df_limpio['Pres'] = pd.to_numeric(df_limpio['Pres'], errors='coerce').clip(lower=850, upper=1100)
    df_limpio['Wspd'] = pd.to_numeric(df_limpio['Wspd'], errors='coerce').clip(lower=0, upper=100)
    df_limpio['Wdir'] = pd.to_numeric(df_limpio['Wdir'], errors='coerce').clip(lower=0, upper=360)
    df_limpio['Snow Depth'] = pd.to_numeric(df_limpio['Snow Depth'], errors='coerce').clip(lower=0, upper=500)
    
    # Aplicar suavizado a los datos
    df_limpio['GHI'] = smooth_data(df_limpio['GHI'].values)
    df_limpio['DNI'] = smooth_data(df_limpio['DNI'].values)
    df_limpio['DHI'] = smooth_data(df_limpio['DHI'].values)
    
    # Rellenar valores nulos mediante interpolación
    df_limpio = df_limpio.interpolate(method='linear')
    
    # Reporte de calidad final
    calidad_final = []
    calidad_final.append("\nValores nulos por columna (después de limpieza):\n")
    calidad_final.append(str(df_limpio.isnull().sum()))
    calidad_final.append("\nEstadísticas descriptivas (después de limpieza):\n")
    calidad_final.append(str(df_limpio.describe()))
    
    # Resumen de correcciones
    resumen = []
    resumen.append("\nResumen de correcciones:")
    resumen.append("-" * 50)
    resumen.append("GHI, DNI, DHI: Valores negativos convertidos a 0, límites aplicados")
    resumen.append("Tdry: Limitada entre -20°C y 50°C")
    resumen.append("Tdew: Limitada entre -25°C y 30°C")
    resumen.append("RH: Limitada entre 0% y 100%")
    resumen.append("Pres: Limitada entre 850 y 1100 hPa")
    resumen.append("Wspd: Limitada entre 0 y 100 m/s")
    resumen.append("Wdir: Limitada entre 0° y 360°")
    resumen.append("Snow Depth: Limitada entre 0 y 500")
    for col in ['GHI', 'DNI', 'DHI', 'Tdry', 'Tdew', 'RH', 'Pres', 'Wspd', 'Wdir', 'Snow Depth']:
        if col in df.columns:
            original_nulls = df[col].isnull().sum()
            final_nulls = df_limpio[col].isnull().sum()
            resumen.append(f"{col}: {original_nulls} nulos antes, {final_nulls} nulos después")
    
    # Guardar el archivo limpio
    df_limpio.to_csv(output_file, index=False)
    print(f"\nArchivo procesado y guardado como: {output_file}")
    
    # Guardar el reporte de calidad
    report_file = output_file.replace('.csv', '_report.txt')
    with open(report_file, 'w') as f:
        f.write('\n'.join(calidad_inicial))
        f.write('\n')
        f.write('\n'.join(resumen))
        f.write('\n')
        f.write('\n'.join(calidad_final))
    print(f"Reporte de calidad guardado como: {report_file}")

def main():
    # Directorio base
    base_dir = '/home/josefa_montoya/Josefamontoya/josefamontoya/PRUEBA_2'
    
    # Lista de archivos a procesar
    files_to_process = [
        'Vallenar_corrupted.csv',
        'calama_corrupted.csv',
        'salvador_corrupted.csv'
    ]
    
    # Procesar cada archivo
    for file_name in files_to_process:
        input_path = os.path.join(base_dir, file_name)
        output_path = os.path.join(base_dir, file_name.replace('_corrupted.csv', '_processed.csv'))
        process_tmy_file(input_path, output_path)

if __name__ == "__main__":
    main() 


Procesando archivo: /home/josefa_montoya/Josefamontoya/josefamontoya/PRUEBA_2/Vallenar_corrupted.csv

Archivo procesado y guardado como: /home/josefa_montoya/Josefamontoya/josefamontoya/PRUEBA_2/Vallenar_processed.csv
Reporte de calidad guardado como: /home/josefa_montoya/Josefamontoya/josefamontoya/PRUEBA_2/Vallenar_processed_report.txt

Procesando archivo: /home/josefa_montoya/Josefamontoya/josefamontoya/PRUEBA_2/calama_corrupted.csv

Archivo procesado y guardado como: /home/josefa_montoya/Josefamontoya/josefamontoya/PRUEBA_2/calama_processed.csv
Reporte de calidad guardado como: /home/josefa_montoya/Josefamontoya/josefamontoya/PRUEBA_2/calama_processed_report.txt

Procesando archivo: /home/josefa_montoya/Josefamontoya/josefamontoya/PRUEBA_2/salvador_corrupted.csv

Archivo procesado y guardado como: /home/josefa_montoya/Josefamontoya/josefamontoya/PRUEBA_2/salvador_processed.csv
Reporte de calidad guardado como: /home/josefa_montoya/Josefamontoya/josefamontoya/PRUEBA_2/salvador_proc